# Groq and Gradio for Image Moderation with LLaVA and Llama Guard 📷

In this tutorial, we'll build an image moderation application powered by Groq for fast image analysis and content safety checking. We'll also leverage [Gradio](https://www.gradio.app/) for creating a user-friendly interface and [Hugging Face Spaces](https://huggingface.co/spaces) for hosting our application.

[Groq](groq.com) is known for crazy fast inference speed that is very well-suited for realtime AI applications, providing multiple Large Language Models (LLMs) across various modalities via Groq API. In this tutorial, we will use [LlaVA 1.5 7B](https://huggingface.co/liuhaotian/llava-v1.5-7b) for image analysis and [Llama Guard 3 8B](https://huggingface.co/meta-llama/Llama-Guard-3-8B) for assessing the safety of a given image.

## Content Moderation
In today's digital age, the internet is a ubiquitous part of our daily lives. However, with the vast amount of information available online, ensuring a safe and positive environment for users has become a significant challenge. This is where content moderation comes into play.

Specifically, image moderation is the process of detecting and filtering out inappropriate or harmful images. This is important for maintaining a safe and positive environment for users, especially in applications like online marketplaces, community forums, and social media in general.

In this tutorial, we will use the LlaVA model to generate a description of a given image and use the generated description as input for the Llama Guard model to check for conteny safety. LlaVA is a state-of-the-art image understanding model that can answer questions about an image, describe its content, and perform Optical Character Recognition (OCR) to extract text from images. Llama Guard is an advanced content moderation tool designed to assess user messages, or text, for harmful content across 14 categories, including hate speech, threats, and misinformation.

Let's dive into the code and learn how to combine the capabilities of Llama Guard and LlaVA to moderate images in real-time!

## Step 1: Create a Free GroqCloud Account and Generate Your Groq API Key

If you don't already have a GroqCloud account, you can create one for free [here](https://console.groq.com) to generate a Groq API Key. We'll need the key to be able to try out the tutorial we build! 

## Step 2: Install and Import Required Libraries
Let's install and import the libraries we'll need to interact with Groq API and build our image moderation application.

In [ ]:
!pip install groq
!pip install gradio

import base64
import io
import gradio as gr
from groq import Groq
from PIL import Image
import requests

## Step 3: Implement Image Encoding Function
Next, we'll implement a function to encode image inputs to base64 format, which is required by Groq API.

In [ ]:
def encode_image(image):
    buffered = io.BytesIO()
    image.save(buffered, format="JPEG")
    return base64.b64encode(buffered.getvalue()).decode('utf-8')

## Step 4: Implement Image Analysis Function
Now let's implement a function that will analyze our input image using the LlaVA model powered by Groq.

In [ ]:
def analyze_image(image, prompt, api_key, is_url=False):
    client = Groq(api_key=api_key)

    if is_url:
        image_content = {"type": "image_url", "image_url": {"url": image}}
    else:
        base64_image = encode_image(image)
        image_content = {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}

    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": prompt},
                        image_content,
                    ],
                }
            ],
            model="llava-v1.5-7b-4096-preview",
        )
        return chat_completion.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"

## Step 5: Implement Content Safety Check Function
Now we'll implement a function to use the Llama Guard model powered by Groq to check the safety of the image by analyzing the image description generated by the LlaVA model.

In [ ]:
def check_content_safety(image_description, api_key):
    client = Groq(api_key=api_key)

    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {"role": "system", "content": "You are a content safety classifier. Analyze the given text and determine if it contains any unsafe or inappropriate content."},
                {"role": "user", "content": f"Please analyze this image description for any unsafe or inappropriate content: {image_description}"}
            ],
            model="llama-guard-3-8b",
        )
        return chat_completion.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"

## Step 6: Implement Image Processing Function
The final function we'll create will handle both uploaded images and image URLs, then call the analysis and safety check functions we built earlier.

In [ ]:
def process_image(image, url, prompt, api_key):
    if image is not None:
        return analyze_image(image, prompt, api_key), check_content_safety(analyze_image(image, prompt, api_key), api_key)
    elif url:
        try:
            response = requests.get(url)
            image = Image.open(io.BytesIO(response.content))
            return analyze_image(url, prompt, api_key, is_url=True), check_content_safety(analyze_image(url, prompt, api_key, is_url=True), api_key)
        except:
            return "Invalid image URL. Please provide a direct link to an image.", ""
    else:
        return "Please provide an image to analyze.", ""

## Step 7: Create Gradio Interface and Launch Application
Now we'll use Gradio components to build out a simple user interface and launch our image moderation application!

In [ ]:
ddef launch():
    with gr.Blocks(
        theme=gr.themes.Default(primary_hue="orange"),
        css="""
        #app-container { max-width: 1000px; margin: auto; padding: 10px; }
        #title { text-align: center; margin-bottom: 10px; font-size: 24px; }
        #groq-badge { text-align: center; margin-top: 10px; }
        .gr-button { border-radius: 15px; }
        .gr-input, .gr-box { border-radius: 10px; }
        .gr-form { gap: 5px; }
        .gr-block.gr-box { padding: 10px; }
        .gr-paddle { height: auto; }
        """
    ) as demo:
        with gr.Column(elem_id="app-container"):
            gr.Markdown("# 🖼️ Groq x Gradio Image Analysis and Content Safety Check", elem_id="title")
            
            with gr.Row():
                api_key = gr.Textbox(label="Groq API Key:", type="password", scale=2)
                prompt = gr.Textbox(
                    label="Image Analysis Prompt:",
                    value="Describe the image content.",
                    scale=3
                )
            
            with gr.Row():
                with gr.Column(scale=1):
                    image_input = gr.Image(type="pil", label="Upload Image:", height=200, sources=["upload"])
                with gr.Column(scale=1):
                    url_input = gr.Textbox(label="Or Paste Image URL:", lines=1)
                    analyze_button = gr.Button("🚀 Analyze Image", variant="primary")
            
            with gr.Row():
                with gr.Column():
                    analysis_output = gr.Textbox(label="Image Analysis with LlaVA 1.5 7B:", lines=6)
                with gr.Column():
                    safety_output = gr.Textbox(label="Safety Check with Llama Guard 3 8B:", lines=6)
            
            analyze_button.click(
                fn=process_image,
                inputs=[image_input, url_input, prompt, api_key],
                outputs=[analysis_output, safety_output]
            )
            
            with gr.Row():
                with gr.Column():
                    gr.HTML("""
                    <div id="groq-badge">
                        <div style="color: #f55036; font-weight: bold; font-size: 1em;">⚡ POWERED BY GROQ ⚡</div>
                    </div>
                    """)
                    
                with gr.Column():
                    gr.Markdown("""
                    **How to use this app:** 
                    1. Enter your [Groq API Key](https://console.groq.com/keys) in the provided field.
                    2. Upload an image file or paste an image URL.
                    3. Use default prompt or enter custom prompt for image analysis.
                    4. Click "Analyze Image" to check for content safety.
                    """)
    
    demo.launch()

if __name__ == "__main__":
    launch()

## (Optional) Step 8: Host on Hugging Face Spaces
If you don't already have one, create a free Hugging Face account [here](https://huggingface.co/join). To deploy our app to Hugging Face Spaces from our browser, all we have to do is drag and drop all related files [here](https://huggingface.co/new-space). In this case, we'll create an `app.py` file as well as a `requirements.txt` file. 

In the `app.py` file, simply copy-paste the above code and feel free to modify for additional features or a different look! 

In the `requirements.txt` file, add in the required dependencies for Hugging Face Spaces to detect and automatically install before deploying our application to a public link that anyone can access.

For this project, the following dependencies were added to the `requirements.txt` file:

```
groq
gradio
```

Once the required application files are added, Hugging Face Spaces will automatically detect, build, run, and deploy our application! You can see and try this tutorial live [here](https://huggingface.co/spaces/Groq/image-moderation)! 😁

# Conclusion
By combining Groq, Gradio, and Hugging Face Spaces, we've created and deployed a powerful and easy-to-use image moderation tool.

We hope you found this tutorial helpful and that you'll use it to create your own image moderation tools and applications. Feel free to experiment with this code and extend the functionality to create your own personal project! Happy building! 🤩